In [1]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Estimator
import matplotlib.pyplot as plt 
import numpy as np 
from qiskit.quantum_info import Statevector, random_clifford 
import os

In [3]:
from tomography import SelfGuidedTomography, Mean_Direct_Fidelity

In [4]:
NQ = 2
MDF = Mean_Direct_Fidelity(NQ)
state = np.random.randn( 2**NQ )
state = state / np.linalg.norm( state )

In [5]:
Omega = random_clifford( NQ ).to_circuit()
state = np.array( Statevector(Omega) )
state

array([0.+0.j        , 0.+0.70710678j, 0.-0.70710678j, 0.+0.j        ])

In [6]:
MDF.Chi( state, None ).round(2)

array([ 0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j,
        0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j,
        0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j])

In [7]:
alpha = 0 #1 / 2**NQ
MDF.Chi( state, truncation=alpha ).round(2)

array([ 0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j,
        0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j,
        0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j])

In [7]:
np.outer( state, state.conj() ).round(2)

array([[ 0.25+0.j  ,  0.  +0.25j,  0.25+0.j  ,  0.  -0.25j],
       [ 0.  -0.25j,  0.25+0.j  ,  0.  -0.25j, -0.25-0.j  ],
       [ 0.25+0.j  ,  0.  +0.25j,  0.25+0.j  ,  0.  -0.25j],
       [ 0.  +0.25j, -0.25+0.j  ,  0.  +0.25j,  0.25+0.j  ]])

### Initial Parameters

In [8]:
NQs = [2,3,4,5,6]
L = len(NQs)
N = 2
Niter = 100
shots = 1000
simulator_ideal=Estimator(backend_options={'shots':shots,
                                            'method':"stabilizer",},
                            transpile_options={'optimization_level':0},
                            ) 

In [13]:
from joblib import Parallel, delayed

def simulate( NQ, simulator ):

    d = 2**NQ

    Omega = random_clifford( NQ ).to_circuit()
    OmegaM=np.outer(np.array(Statevector(Omega)),
                    (np.array(Statevector(Omega))).conj()) 

    psi0 = np.random.rand(d) + 1j * np.random.rand(d)
    psi0 = psi0 / np.linalg.norm(psi0)
    guess = psi0

    MDF = Mean_Direct_Fidelity(NQ)

    stop_measuring = lambda x : ( np.linalg.norm( list( x.values() ) ) > 0.98 )

    I_ex = lambda x : 1 - MDF.MeanFidelity(1, 
                                            2*NQ**2, 
                                            x,
                                            Omega,
                                            simulator,
                                            truncation = None,
                                            stop_measuring = stop_measuring,  )
    
    I_th = lambda x: 1 - np.vdot( x, OmegaM@x )/(np.linalg.norm( x ))**2

    Fidelities = []
    Measures = []
    Last=[]
    norms = []
    def callback( i, x ):
        Last.append(x)
        Fidelities.append( I_th(x) )
        Measures.append(MDF.num_exp_meas)
        norms.append( np.linalg.norm( list( MDF.Measures.values() ) ) )
        return None

    SelfGuidedTomography( I_ex, 
                            guess, 
                            num_iter=1000, 
                            callback = callback,
                            )
    
    Results=[np.array(Fidelities), 
                np.array(Measures), 
                np.array(Last),
                np.array( norms )
                ] 

    return Results  #j index the average over the Hilbert space 


In [14]:
from joblib import Parallel, delayed

iterator = np.repeat(NQs, N).flatten()

R = Parallel(n_jobs=-1, verbose=11)(delayed(simulate)(int(NQ)) for NQ in iterator)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    0.4s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.4s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.8s
[Pa

In [15]:
Fids, Meas, _ = [[[R[i + j * N][r] for i in range(N)] for j in range(len(NQs))]
                   for r in range(3)]

Fids = np.real(Fids)
Meas = np.array(Meas)

In [ ]:
# Save data. Ax order Fids and Meas are: [NQ, Nrun, Niter]

filename = "sgqt_without_truncation.npz"

data = {
    "N": [N],
    "shots": [shots],
    "Niter": [Niter],
    "NQs": NQs,
    "Fids": Fids,
    "Meas": Meas,
    "info": ["N, shots, and Niter are single integers in one array each.",
             "NQs is the array of number of qubits.",
             "Fids and Meas have shapes (len(NQs), N, Niter)"],
}

# Make directory "data" if not present
os.makedirs("data", exist_ok=True)

# Save
filepath = os.path.join("data", filename)
np.savez(filepath, **data)